<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Catboost" data-toc-modified-id="Catboost-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Catboost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip install catboost
!pip install langid

   ---------------------------------------- 0.0/101.8 MB ? eta -:--:--
   ---------------------------------------- 0.8/101.8 MB 8.3 MB/s eta 0:00:13
   - -------------------------------------- 2.9/101.8 MB 10.5 MB/s eta 0:00:10
   -- ------------------------------------- 5.2/101.8 MB 10.6 MB/s eta 0:00:10
   -- ------------------------------------- 7.6/101.8 MB 10.9 MB/s eta 0:00:09
   --- ------------------------------------ 9.7/101.8 MB 10.6 MB/s eta 0:00:09
   ---- ----------------------------------- 12.1/101.8 MB 10.9 MB/s eta 0:00:09
   ----- ---------------------------------- 14.7/101.8 MB 11.3 MB/s eta 0:00:08
   ------ --------------------------------- 16.8/101.8 MB 11.2 MB/s eta 0:00:08
   ------- -------------------------------- 19.1/101.8 MB 11.1 MB/s eta 0:00:08
   -------- ------------------------------- 21.8/101.8 MB 11.3 MB/s eta 0:00:08
   --------- ------------------------------ 24.9/101.8 MB 11.5 MB/s eta 0:00:07
   ---------- ----------------------------- 27.5/101.8 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
stopwords = list(set(nltk_stopwords.words('russian')))

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)

Matplotlib is building the font cache; this may take a moment.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/toxic_comments.csv'

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.toxic.value_counts()

Можно отметить высокую степень дисбаланса классов.

Напишем функцию, которая будет возвращать язык переданного ей текста, и применим ее к столбцу с комментариями

In [ ]:
def language_detector(row):
  return langid.classify(row['text'])[0]

df['language'] = df.apply(language_detector, axis=1)

In [ ]:
df.head()

In [ ]:
df.language.value_counts()

Как видно, язык написания абсолютного большинства комментариев определен как англйиский.

Напишем функцию, которая очищаяет и лемматизирует текст.

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_text(row):
  clear_text = ' '.join(re.sub(r'[^a-zA-Z ]', ' ', row['text']).split())
  tokens = nltk.word_tokenize(clear_text)
  tokens_lemmatized = []
  for token in tokens:
    tokens_lemmatized.append(lemmatizer.lemmatize(token))
  return str.lower(' '.join(tokens_lemmatized))  

In [ ]:
df['lemmas'] = df.apply(lemmatize_text, axis=1)

In [ ]:
df.head()

In [ ]:
features = df['lemmas']
target = df['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.1, random_state=1)

Определим набор стоп-слов, которые будут исключены при векторизации текста

In [ ]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

Обучим векторайзер на обучающей выборке, затем преобразуем им обучающую и тестовую выборки

In [ ]:
tf_idf = TfidfVectorizer(stop_words=stopwords)

train_features_tf_idf = tf_idf.fit_transform(features_train)
test_features_tf_idf = tf_idf.transform(features_test)

## Обучение

### Логистическая регрессия

In [ ]:
model = LogisticRegression()
model.fit(train_features_tf_idf, target_train)
predictions_logistic = model.predict(test_features_tf_idf)
print('F1-мера для модели логистической регрессии составила {:.2f}'.format(
    f1_score(target_test, predictions_logistic)))

Ранее мы обратили внимание, что классы в датасете несбалансированы. Попробуем обучить модель со взвешенными классами.

In [ ]:
model = LogisticRegression(class_weight='balanced', max_iter=300)
model.fit(train_features_tf_idf, target_train)
predictions_logistic_balanced_weight = model.predict(test_features_tf_idf)
print('F1-мера для модели логистической регрессии со взвешенными классами составила {:.2f}'.format(
    f1_score(target_test, predictions_logistic_balanced_weight)))

### Catboost

In [ ]:
features_cb = df[['lemmas']]
target_cb = df[['toxic']]

Поделим выборки на обучающую, валидационную и тестовую.

In [ ]:
features_cb_train, features_cb_, target_cb_train, target_cb_ = train_test_split(features_cb, target_cb, test_size=.2, random_state=1)
features_cb_valid, features_cb_test, target_cb_valid, target_cb_test = train_test_split(features_cb_, target_cb_, test_size=.5, random_state=1)

In [ ]:
%%time
model = CatBoostClassifier(iterations=500, learning_rate=0.03, depth=10)
model.fit(features_cb_train, target_cb_train, 
          eval_set=(features_cb_valid, target_cb_valid),
          text_features=['lemmas'], verbose=50)

In [ ]:
predictions_cb = model.predict(features_cb_test)
print('F1-мера для модели CatBoost составила {:.2f}'.format(
    f1_score(target_cb_test, predictions_cb)))

## Выводы

Выполнен анализ работы моделей логистической регрессии и CatBoost на классификации текстовых комментариев. Модель логистической регрессии показала более слабый результат, но нужно отметить высокую скорость ее работы. Модель CatBoost показала хороший результат, но потребовала значительные траты времени и ресурсов на обучение. Помимо этого, преимущество модели CatBoost заключается в том, что она работает непосредственно с текстовыми признаками, без необходимости в их векторизации.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны